### Multi-omic Dataset: Latent representation using Autoencoders
- Load Data
- Normalize Data
- Define Autoencoder Model
- Train Autoencoder with normalized dataset
- Use transformed dataset for classification
- Use transformed dataset for clustering
- Evaluation and Conclusions

In [5]:
import numpy as np
import random as rn
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

rn.seed(1)
np.random.seed(1)
tf.random.set_seed(1)
tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)

# Load Data
X_renal_data = pd.read_csv('./x_exp_renal.csv', sep='\t') # Dataset has Donor ID as first column
y_renal_data = pd.read_csv('./y_renal.csv', sep=',') # Dataset has Donor ID on first column and Label on second column.

from sklearn.model_selection import train_test_split
# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_renal_data.iloc[:,1:],y_renal_data["label"],test_size=0.2, random_state=1) # Drop the Donor ID column from both datasets
# Normalization of data sets
# Data Scaling MinMax
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train_norm = X_train
X_test_norm = X_test

X_train_norm = pd.DataFrame(scaler.fit_transform(X_train_norm))
X_test_norm = pd.DataFrame(scaler.transform(X_test_norm))

# We will use "X_train_norm" as training dataset for the Autoencoder

FileNotFoundError: [Errno 2] File b'./x_exp_renal.csv' does not exist: b'./x_exp_renal.csv'

In [4]:
## AUTOENCODER MODEL
# Define the model using the keras functional API
def build_autoencoder(encoding_dim: int, number_features: int, regularizer: tf.keras.regularizers.Regularizer, dropout: float):
    """Two-input autoencoder build function
       Parameters: encoding_dim: Size of the latent space (bottleneck layer size).
                   number_features: Tuple with the sizes of the two inputs.
                   regularizer: keras regularizer object
       Returns the 3 models: full autoencoder, the encoder part and the decoder part
    """
    if dropout > 1:
        dropout = 1
    elif dropout < 0:
        dropout = 0
    # this is the reduction of our encoded representations, in times.
    print(f"Compression: {number_features/encoding_dim}")

    first_layer_size = number_features/4
    second_layer_size = number_features/12
    
    ## ENCODER
    # encoder first input placeholder.
    first_input = layers.Input(shape=(number_features))
    # encoder first Hidden Layer - H1
    H1 = layers.Dense(first_layer_size, activation='relu', kernel_regularizer=regularizer)(first_input)
    # encoder first Dropout Layer - D1
    D1 = layers.Dropout(dropout)(H1)
    # encoder first Batch Normalization Layer - BN1
    BN1 = layers.BatchNormalization()(D1)
    # encoder second Hidden Layer - H2
    H2 = layers.Dense(second_layer_size, activation='relu', kernel_regularizer=regularizer)(BN1)
    # encoder second Dropout Layer - D2
    D2 = layers.Dropout(dropout)(H2)
    # encoder first path second Batch Normalization Layer - BN2
    BN2 = layers.BatchNormalization()(D2)

   
    ## BOTTLENECK 
    bottleneck = layers.Dense(encoding_dim, activation='relu', kernel_regularizer=regularizer)(BN2)

    # this model maps an input to its encoded representation
    encoder = keras.models.Model(first_input, bottleneck, name='encoder')

    ## DECODER
    # Decoder Input Layer - Encoding dimension
    encoded_input = layers.Input(shape=(encoding_dim,))
    # decoder first Dropout Layer - D3
    D3 = layers.Dropout(dropout)(encoded_input)
    # decoder first Batch Normalization Layer - BN3 
    BN3 = layers.BatchNormalization()(D3)
    # decoder first Hidden Layer - H3
    H3 = layers.Dense(second_layer_size, activation='relu', kernel_regularizer=regularizer)(BN3)
    # decoder second Dropout Layer - D4
    D4 = layers.Dropout(dropout)(H3)
    # decoder second Batch Normalization Layer - BN4 
    BN4 = layers.BatchNormalization()(D4)
    # decoder reconstruction layer - O1
    O1 = layers.Dense(number_features, activation='sigmoid')(BN4)

    # create the decoder model
    decoder = keras.models.Model(encoded_input, O1)

    # create the full autoencoder
    encoder_model = encoder(first_input)
    decoder_model = decoder(encoder_model)

    autoencoder = keras.models.Model(first_input, decoder_model, name="autoencoder")
    
    return autoencoder, encoder, decoder

# Set Optimizer: Adam with learning rate=0.001
optimizer = tf.keras.optimizers.Adam(0.001)
# Set Early Stop Callback
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=10,  mode='auto', baseline=None, restore_best_weights=False, verbose=1)

## Call autoencoder build function and get the AE, the encoder and the decoder.
autoencoder, encoder, decoder = build_autoencoder(encoding_dim=20, number_features=X_train_norm.shape[1], regularizer=tf.keras.regularizers.l1_l2(0.0001,0), dropout=0.5)
# Compile the autoencoder using Mean Square Error loss function.
autoencoder.compile(optimizer=optimizer,
                        loss="mse",
                        metrics=['mse'])


Compression: 1025.1


In [ ]:
## TRAINING
# Fit the training data into the autoencoder.
history = autoencoder.fit(X_train_norm,X_train_norm,
                          validation_data=(X_test_norm,X_test_norm)
                          epochs=200,
                          verbose=0,
                          callbacks=[early_stop])
# Plot training vs validation losses
plt.plot(history.history["loss"], c = 'b', label = "Training")
plt.plot(history.history["val_loss"], c = 'r', label = "Validation")
plt.title("Autoencoder Loss during training epochs")
plt.legend()
plt.show()
print(history.history["loss"][-1])

# Encode datasets using the trained encoder.
X_train_encoded = encoder.predict(X_train_norm)
X_test_encoded = encoder.predict(X_test_norm)

# Renormalize data
scaler = MinMaxScaler()
X_train_encoded = pd.DataFrame(scaler.fit_transform(X_train_encoded))
X_test_encoded = pd.DataFrame(scaler.fit_transform(X_test_encoded))

In [ ]:
### CLASSIFICATION ###
# We use the reduced dataset to train a classifier and compare it against the same classifier trained with the original dataset.

# One hot encode labels
from sklearn.preprocessing import LabelEncoder
OH_encoder = LabelEncoder()
OH_y_train = pd.DataFrame(OH_encoder.fit_transform(y_train))
OH_y_test = pd.DataFrame(OH_encoder.transform(y_test))
y_train_oh = keras.utils.to_categorical(OH_y_train)
y_test_oh = keras.utils.to_categorical(OH_y_test)

## Definition of the best classifier obtained previously (CTG_dataset_classification)
def build_best_model(dropout: int, l1: int, l2: int, input_shape: int):
    model = keras.Sequential([
        layers.Dense(100, activation=tf.nn.relu ,kernel_regularizer=keras.regularizers.l1_l2(l1,l2), input_shape=input_shape),
        layers.Dropout(dropout),
        layers.BatchNormalization(),  
        layers.Dense(25,activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l1_l2(l1,l2)),
        layers.Dropout(dropout),
        layers.BatchNormalization(),
        layers.Dense(10,activation=tf.nn.relu, kernel_regularizer=keras.regularizers.l1_l2(l1,l2)),
        layers.Dropout(dropout),
        layers.BatchNormalization(),
        layers.Dense(2,activation=tf.nn.softmax)
      ])

    optimizer = tf.keras.optimizers.Adam(0.001)

    model.compile(loss='categorical_crossentropy',
                optimizer=optimizer,
                metrics=['accuracy'])
    return model


# Fit best model with dimensionality reduction data
model_ae = build_best_model(0.5,0.0001, 0.00001, X_train_encoded.shape[1])
history_ae = model_ae.fit(X_train_pca, y_train_oh, epochs=150,
                    validation_split = 0.1, verbose=0, callbacks=[early_stop], shuffle=False)
hist_pca = pd.DataFrame(history_ae.history)

test_loss, test_acc = model_pca.evaluate(X_test_pca, y_test_oh)

# Fit best model with concatenated data
model = build_best_model(0.00001, X_train_norm.shape[1])
history = model.fit(X_train, y_train_oh, epochs=150,
                    validation_split = 0.1, verbose=0, callbacks=[early_stop])
hist = pd.DataFrame(history.history)

test_loss, test_acc = model.evaluate(X_test, y_test_oh)